# EyeSpy

In [142]:
import os
import numpy as np
import cv2
import time as tm
import argparse
from imageai import Detection

In [105]:
execution_path = os.getcwd()
VID_PATH = './demos/trial1.mp4'
CONTINUITY_THRESHOLD = 8 #For cutting out boxes

MIN_SECONDS = 1 # (seconds) Minimum duration of a moving object
INTERVAL_BW_DIVISIONS = 5 # (seconds) For distributing moving objects over a duration to reduce overlapping.
GAP_BW_DIVISIONS = 1.5 #(seconds)
FRAMES_PER_DETECTION = 3

In [120]:
fgbg = cv2.createBackgroundSubtractorKNN()

detector = Detection.ObjectDetection()
detector.setModelTypeAsTinyYOLOv3()
detector.setModelPath("./yolo-tiny.h5")
detector.loadModel()
custom_objects = detector.CustomObjects(car=True, motorcycle=True, person=True,  bicycle=True, bus=True, truck=True)

### Sample output of detector

In [7]:
# detections = detector.detectCustomObjectsFromImage(custom_objects=custom_objects, input_image=os.path.join(execution_path , "rec.jpg"), output_image_path=os.path.join(execution_path , "test.jpg"), minimum_percentage_probability=31)
detected_image_array, detections = detector.detectCustomObjectsFromImage(
                                        output_type='array',
                                        custom_objects=custom_objects, 
                                        input_image=os.path.join(execution_path , "rec.jpg"), 
                                        minimum_percentage_probability=45
                                    )
frame = cv2.imread('rec.jpg')
for eachObject in detections:
    rect = eachObject["box_points"]
    cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")
cv2.imshow('frame', detected_image_array)
cv2.waitKey(0)
cv2.destroyAllWindows()

car  :  73.92616271972656  :  [320, 144, 339, 164]
--------------------------------
car  :  93.39035749435425  :  [529, 229, 573, 267]
--------------------------------


## Detection on video

In [160]:
vid = cv2.VideoCapture(VID_PATH)
fps = int(vid.get(cv2.CAP_PROP_FPS))
frames = []
ind = 0
start_time = tm.time()
all_conts = []
all_types = []
while(vid.isOpened()):
    ret,frame = vid.read()
    if(not ret): break;
    if(ind%FRAMES_PER_DETECTION == 0):
        frames.append(frame)
        detected_image_array, detections = detector.detectCustomObjectsFromImage(
            input_type="array",
            output_type='array',
            custom_objects=custom_objects,
            input_image=frame, 
            minimum_percentage_probability=50
        )
        conts = np.zeros((len(detections),4)) 
        types = []; i=0
        for eachObject in detections:
            rect = eachObject["box_points"]
            conts[i] = np.array([rect[0],rect[1],rect[2]-rect[0],rect[3]-rect[1]])
            types.append(eachObject["name"])
            i += 1
        print(ind//FRAMES_PER_DETECTION, end=' ')
        all_conts.append(conts)
        all_types.append(types)
        cv2.imshow('frame', detected_image_array)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    ind += 1
frames = np.array(frames)
print("--- %s seconds ---" % (tm.time() - start_time))
vid.release()
cv2.destroyAllWindows()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 --- 119.55718755722046 seconds ---


## Object tracking 

In [161]:
def get_centres(p1):
    return np.transpose(np.array([p1[:,0] + p1[:,2]/2, p1[:,1] + p1[:,3]/2]))

In [162]:
def distance(p1, p2):
    p1 = np.expand_dims(p1, 0)
    if p2.ndim==1:
        p2 = np.expand_dims(p2, 0)
        
    c1 = get_centres(p1)
    c2 = get_centres(p2)
    return np.linalg.norm(c1 - c2, axis=1)

def get_nearest(p1, points):
    """returns index of the point in *points* that is closest to p1"""
    return np.argmin(distance(p1, points)), np.min(distance(p1, points))

In [163]:
class box:
    def __init__(self, coords, time, otype):
        self.coords = coords #coordinates
        self.time   = time #nth frame/time
        self.type   = otype
        
class moving_obj:
    def __init__(self, starting_box):
        self.boxes = [starting_box]
        self.type = None
        self.img = None
    
    def add_box(self, box):
        self.boxes.append(box)
    
    def last_coords(self):
        return self.boxes[-1].coords
    
    def age(self, curr_time):
        last_time = self.boxes[-1].time
        return curr_time - last_time    

In [174]:
"""Will associate boxes into objects"""
#old - boxes in the previous frame
#new - boxes in the current frame


moving_objs = []

for curr_time, new_boxes in enumerate(all_conts): #iterating over frames
    if len(new_boxes) != 0: #if not empty
        new_assocs = [None]*len(new_boxes) #all new boxes initially are not associated with any moving_objs
        obj_coords = np.array([obj.last_coords() for obj in moving_objs if obj.age(curr_time)<CONTINUITY_THRESHOLD])
        unexp_idx = -1 #index of unexpired obj in moving_objs
        for obj_idx, obj in enumerate(moving_objs):
            if obj.age(curr_time) < CONTINUITY_THRESHOLD: #checking only unexpired objects
                unexp_idx += 1
                nearest_new, dst1 = get_nearest(obj.last_coords(), new_boxes) #nearest box to obj
                nearest_obj, dst = get_nearest(new_boxes[nearest_new], obj_coords) #nearest obj to box

                if nearest_obj==unexp_idx: #both closest to each-other
                    #associate
                    if(dst < 100):
                        new_assocs[nearest_new] = obj_idx
    
    
    for new_idx, new_coords in enumerate(new_boxes):
        new_assoc = new_assocs[new_idx]
        new_box = box(new_coords, curr_time, all_types[curr_time][new_idx])

        if new_assoc is not None: 
            #associate new box to moving_obj
            moving_objs[new_assoc].add_box(new_box)
        else: 
            #add a fresh, new moving_obj to moving_objs
            new_moving_obj = moving_obj(new_box)
            moving_objs.append(new_moving_obj)
print("Done")

Done


In [169]:
#Removing objects that occur for a very small duration

MIN_FRAMES = MIN_SECONDS*(fps//3)

moving_objs = [obj for obj in moving_objs if (obj.boxes[-1].time-obj.boxes[0].time)>=MIN_FRAMES]
len(moving_objs)

56

## Displaying result for an object

In [187]:
IND = 27
boxes = moving_objs[IND].boxes
outv = frames.copy()
for rect in boxes:
    x,y,w,h = map(int, list(rect.coords))
    cv2.rectangle(outv[rect.time], (x,y), (x+w,y+h), (0,255,0), 2)
out = cv2.VideoWriter('out_summary.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, (800,480))
for frame in outv:
    cv2.imshow('Video summary',frame)
    out.write(frame)
    tm.sleep(0.01)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    tm.sleep(1/30) #TODO: FPS
out.release()
cv2.destroyAllWindows()

## Setting type and image for objects

In [188]:
for i,obj in enumerate(moving_objs):
    boxes = obj.boxes
    g = 0
    mx = 0
    otype = {}
    for rect in boxes:
        x,y,w,h = map(int, list(rect.coords))
        area = w*h
        if(area > mx):
            mx = area
            mx_i = g
            cv2.imwrite('./test/'+str(i)+'_'+str(g)+'.jpg', frames[rect.time, y:y+h, x:x+w])
        if(rect.type not in otype): otype[rect.type] = 0;
        otype[rect.type] += 1
        g += 1
    moving_objs[i].type = max(otype, key=otype.get)
    rect = boxes[mx_i]
    x,y,w,h = map(int, list(rect.coords))
    cv2.imwrite('./result/'+str(i)+'.jpg', frames[rect.time, y:y+h, x:x+w])
    moving_objs[i].img = frames[rect.time, y:y+h, x:x+w]

## Color Detection

In [135]:
sample = cv2.imread('./result/2.jpg')
avg2 = np.float32(sample) #BG-Ext
cv2.accumulateWeighted(sample, avg2, 0.01)
fgmask = fgbg.apply(sample)
# sample = cv2.cvtColor(sa, cv2.COLOR_BGR2GRAY);

contours = cv2.findContours(fgmask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
img = cv2.drawContours(fgmask, contours[1], -1, (0,255,0), 3)

cv2.imshow('asd', img)
cv2.waitKey(0)
cv2.destroyAllWindows()